In [51]:
# Harrington Listing 11.1
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return map(frozenset, C1)
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not ssCnt.has_key(can): ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [52]:
# Harrington Listing 11.2
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2:
                retList.append(Lk[i] | Lk[j])
    return retList
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = map(set, dataSet)
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k=2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [53]:
# Harrington Listing 11.3
def generateRules(L, supportData, minConf=0.7):
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList,\
                                minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList
        
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        if conf >= minConf:
            print freqSet-conseq,'-->',conseq,'conf:',conf
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)):
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [54]:
import re

# Import dataset
dataSet = []
for line in open('votesmart_20bill.txt'):
    if '(lp' in line:
        dataSet.append([])
    elif len(re.sub(r"\D", "", line)) > 0:
        dataSet[-1].append(int(re.sub(r"\D", "", line)))
dataSet = [d for d in dataSet if len(d) > 0]

pattern = re.compile("S'(.*)'")
# Import rule meanings
itemMeaning = []
for line in open('votesmart_20bill_meaning.txt'):
    for match in re.finditer(pattern, line):
        itemMeaning.append(match.groups()[0])

In [55]:
for i in range(5):
    L, suppData = apriori(dataSet, minSupport=0.5-i*0.05)
    print len(L), [len(l) for l in L]

3 [4, 1, 0]
3 [9, 2, 0]
4 [14, 24, 9, 0]
7 [15, 48, 68, 48, 16, 2, 0]
8 [21, 69, 104, 108, 77, 32, 6, 0]


In [56]:
rules = generateRules(L,suppData)

frozenset([15]) --> frozenset([1]) conf: 0.961538461538
frozenset([1]) --> frozenset([15]) conf: 0.827205882353
frozenset([15]) --> frozenset([21]) conf: 0.897435897436
frozenset([22]) --> frozenset([1]) conf: 0.951351351351
frozenset([3]) --> frozenset([25]) conf: 0.94298245614
frozenset([25]) --> frozenset([3]) conf: 0.80223880597
frozenset([3]) --> frozenset([26]) conf: 0.872807017544
frozenset([26]) --> frozenset([3]) conf: 0.708185053381
frozenset([17]) --> frozenset([1]) conf: 0.87037037037
frozenset([1]) --> frozenset([17]) conf: 0.863970588235
frozenset([29]) --> frozenset([1]) conf: 0.981308411215
frozenset([1]) --> frozenset([29]) conf: 0.772058823529
frozenset([17]) --> frozenset([21]) conf: 0.92962962963
frozenset([7]) --> frozenset([25]) conf: 0.912698412698
frozenset([25]) --> frozenset([7]) conf: 0.858208955224
frozenset([22]) --> frozenset([8]) conf: 0.962162162162
frozenset([8]) --> frozenset([22]) conf: 0.941798941799
frozenset([25]) --> frozenset([0]) conf: 0.8395522

In [57]:
rules = generateRules(L,suppData, minConf=0.95)

frozenset([15]) --> frozenset([1]) conf: 0.961538461538
frozenset([22]) --> frozenset([1]) conf: 0.951351351351
frozenset([29]) --> frozenset([1]) conf: 0.981308411215
frozenset([22]) --> frozenset([8]) conf: 0.962162162162
frozenset([3]) --> frozenset([9]) conf: 1.0
frozenset([29]) --> frozenset([11]) conf: 0.96261682243
frozenset([3]) --> frozenset([7]) conf: 0.973684210526
frozenset([9]) --> frozenset([7]) conf: 0.963265306122
frozenset([8]) --> frozenset([1]) conf: 0.978835978836
frozenset([17]) --> frozenset([13]) conf: 0.962962962963
frozenset([3]) --> frozenset([23]) conf: 0.964912280702
frozenset([23]) --> frozenset([7]) conf: 0.95
frozenset([14]) --> frozenset([21]) conf: 0.953608247423
frozenset([23]) --> frozenset([0]) conf: 0.954166666667
frozenset([11]) --> frozenset([13]) conf: 0.961832061069
frozenset([28]) --> frozenset([21]) conf: 0.96335078534
frozenset([9]) --> frozenset([0]) conf: 0.983673469388
frozenset([29]) --> frozenset([13]) conf: 0.985981308411
frozenset([8])

In [58]:
rules = generateRules(L,suppData, minConf=0.99)

frozenset([3]) --> frozenset([9]) conf: 1.0
frozenset([3]) --> frozenset([0]) conf: 0.995614035088
frozenset([3]) --> frozenset([0, 9]) conf: 0.995614035088
frozenset([3, 7]) --> frozenset([0, 9]) conf: 0.995495495495
frozenset([26, 3]) --> frozenset([0, 9]) conf: 1.0
frozenset([3, 4]) --> frozenset([0, 9]) conf: 0.995348837209
frozenset([3, 23]) --> frozenset([0, 9]) conf: 0.995454545455
frozenset([25, 3]) --> frozenset([0, 9]) conf: 0.995348837209
frozenset([25, 3, 23]) --> frozenset([0, 9]) conf: 0.995169082126
frozenset([26, 3, 4]) --> frozenset([0, 9]) conf: 1.0
frozenset([25, 3, 4]) --> frozenset([0, 9]) conf: 0.99504950495
frozenset([25, 3, 7]) --> frozenset([0, 9]) conf: 0.995215311005
frozenset([3, 4, 7]) --> frozenset([0, 9]) conf: 0.995238095238
frozenset([26, 3, 23]) --> frozenset([0, 9]) conf: 1.0
frozenset([23, 3, 7]) --> frozenset([0, 9]) conf: 0.995348837209
frozenset([25, 26, 3]) --> frozenset([0, 9]) conf: 1.0
frozenset([3, 4, 23]) --> frozenset([0, 9]) conf: 0.995192

In [59]:
itemMeaning[26]

'Prohibiting the Use of Federal Funds for NASCAR Sponsorships -- Nay'

In [60]:
itemMeaning[3]

'Prohibiting Federal Funding of National Public Radio -- Yea'

In [61]:
itemMeaning[4]

'Removing Troops from Afghanistan -- Nay'

In [62]:
itemMeaning[23]

'Prohibiting Use of Federal Funds For Planned Parenthood -- Yea'

In [63]:
itemMeaning[25]

'Reducing Federal Funding of the US Institute of Peace -- Yea'

In [64]:
itemMeaning[7]

'Terminating the Home Affordable Modification Program -- Yea'